# stock market api project
Rapid API URL -https://rapidapi.com/alphavantage/api/alpha-vantage

In [3]:
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [3]:
from dotenv import load_dotenv
load_dotenv()

False

In [1]:
import requests

url = "https://alpha-vantage.p.rapidapi.com/query"

querystring = {"datatype":"json","keywords":"microsoft","function":"SYMBOL_SEARCH"}

headers = {
	"x-rapidapi-key": "f3f36ba864msh2873b1571d1dfb5p14ec18jsn1026ae5ea1a9",
	"x-rapidapi-host": "alpha-vantage.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

response.json()

{'bestMatches': [{'1. symbol': 'MSF0.FRK',
   '2. name': 'MICROSOFT CORP. CDR',
   '3. type': 'Equity',
   '4. region': 'Frankfurt',
   '5. marketOpen': '08:00',
   '6. marketClose': '20:00',
   '7. timezone': 'UTC+02',
   '8. currency': 'EUR',
   '9. matchScore': '0.6429'},
  {'1. symbol': 'MSFT',
   '2. name': 'Microsoft Corporation',
   '3. type': 'Equity',
   '4. region': 'United States',
   '5. marketOpen': '09:30',
   '6. marketClose': '16:00',
   '7. timezone': 'UTC-04',
   '8. currency': 'USD',
   '9. matchScore': '0.6154'},
  {'1. symbol': '0QYP.LON',
   '2. name': 'Microsoft Corporation',
   '3. type': 'Equity',
   '4. region': 'United Kingdom',
   '5. marketOpen': '08:00',
   '6. marketClose': '16:30',
   '7. timezone': 'UTC+01',
   '8. currency': 'USD',
   '9. matchScore': '0.6000'},
  {'1. symbol': 'MSF.DEX',
   '2. name': 'Microsoft Corporation',
   '3. type': 'Equity',
   '4. region': 'XETRA',
   '5. marketOpen': '08:00',
   '6. marketClose': '20:00',
   '7. timezone': '